
## Трансформеры

## BPE


**Проблема**

 - Токенизация слов слишком высокоуровнева
 - Токенизация по буквам слишком низкоуровнева


 **Решение**

 Будем соединять токены, пока не достигнем определенного числа.

![](https://lena-voita.github.io/resources/lectures/seq2seq/bpe/build_merge_table.gif)

In [1]:
!pip install -q youtokentome
!pip install -q datasets

     |████████████████████████████████| 1.7MB 22.6MB/s 
     |████████████████████████████████| 163kB 22.3MB/s 
     |████████████████████████████████| 245kB 34.5MB/s 
     |████████████████████████████████| 17.7MB 207kB/s 


In [2]:
from datasets import load_dataset
from youtokentome import BPE

In [3]:
dataset = load_dataset("ag_news")

Using custom data configuration default


Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a. Subsequent calls will reuse this data.


In [4]:
with open("data.txt", "w") as out:
    for data_d in dataset["train"]:
        out.write(data_d["text"]+"\n")

In [5]:
data_d['text']

'Nets get Carter from Raptors INDIANAPOLIS -- All-Star Vince Carter was traded by the Toronto Raptors to the New Jersey Nets for Alonzo Mourning, Eric Williams, Aaron Williams, and a pair of first-round draft picks yesterday.'

In [6]:
bpe = BPE.train(data="data.txt", vocab_size=5000, model="bpe.bin")

In [7]:
encoded = bpe.encode("Hello there! What is BPE?")

In [8]:
tokenized = ""
for word in encoded:
    tokenized += " " + bpe.id_to_subword(word)
tokenized

' ▁H ell o ▁there ! ▁What ▁is ▁B P E ?'

## GPT 2


### Encoder vs Decoder

Так в чем разница?


![](http://jalammar.github.io/images/gpt2/self-attention-and-masked-self-attention.png)

## Авторегрессионный трансформер

![](http://jalammar.github.io/images/xlnet/gpt-2-autoregression-2.gif)

## Huggingface's transformers

![](https://raw.githubusercontent.com/huggingface/transformers/master/docs/source/imgs/transformers_logo_name.png)

In [9]:
!pip install -q transformers

     |████████████████████████████████| 1.5MB 17.2MB/s 
     |████████████████████████████████| 2.9MB 58.7MB/s 
     |████████████████████████████████| 890kB 40.0MB/s 


In [10]:
# import torch

from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer

In [11]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
ru_gpt = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2", output_attentions=True)

In [12]:
device = "cuda"
ru_gpt.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [15]:
start = tokenizer.encode(
    "месяц май  ", 
    return_tensors="pt"
)

result = ru_gpt.generate(
    start.to(device), 
    max_length=30,
    num_beams=10,
    early_stopping=True, 
    no_repeat_ngram_size=3,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [16]:
print(tokenizer.decode(result.cpu().flatten().numpy(), skip_special_tokens=True))

месяц май   Оригинал записи и комментарии на LiveInternet.ru


20750750	asya-cher	2017


In [18]:
inp = """
20 законов чего-нибудь
"""

encoded = tokenizer.encode(inp, return_tensors="pt")


result = ru_gpt.generate(
    encoded.to(device),
    max_length=1000,
    early_stopping=True,
    no_repeat_ngram_size=5)

print(tokenizer.decode(result.cpu().flatten().numpy(), skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



20 законов чего-нибудь




20 законов чего угодно



1. Закон притяжения

Если вы хотите, чтобы к вам притягивались люди, начните с себя.

2. Закон притяжения работает, если вы хотите, чтобы люди к вам притягивались.

3. Закон притяжения действует, если вы хотите привлечь к себе людей.

4. Закон притяжения не работает, если вы не хотите привлекать к себе людей. Вы можете привлечь к себе людей, но не можете заставить их к вам притягиваться.

5. Закон притяжения — это не то, что вы думаете.

6. Закон притяжения может работать, если вы хотите заставить людей к вам притягиваться, но не можете их заставить к вам притягиваться сами.

7. Закон притяжения никогда не работает, если люди не хотят притягиваться к вам.

8. Закон притяжения всегда работает, если вы можете заставить людей к вам притянуться.

9. Закон притяжения, как и закон притяжения, не работает, если не хотите, чтобы люди притягивались к вам.
Вы можете заставить людей к себе притягиваться, но вы не можете заставить их притягиват

# Полезные ссылки

1. https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html
2. http://jalammar.github.io/illustrated-transformer/
3. http://jalammar.github.io/illustrated-gpt2/
